In [37]:
import os
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.plotting import output_notebook, output_file, reset_output
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler

from extractor import embedding_extractor

In [26]:
target_gender = ['female','male']
target_inst = ['vocal','synth','strings','quiet']
target_genre = ['classical','electronic','rock','country','metal']

In [41]:
dataset_path = "dataset/mtat"
# train_list = np.load(os.path.join(dataset_path, "split", "train.npy"))
# binary = np.load(os.path.join(dataset_path, "split", "binary.npy"))
# labels = np.load(os.path.join(dataset_path, "split", "tags.npy"))
df_annotation = pd.read_csv(os.path.join(dataset_path, "annotations_final.csv"), sep="\t")

In [64]:
female_mp3path = list(df_annotation[df_annotation["female"] != 0]["mp3_path"])[:100]
male_mp3path = list(df_annotation[df_annotation["male"] != 0]["mp3_path"])[:100]
strings_mp3path = list(df_annotation[df_annotation["strings"] != 0]["mp3_path"])[:100]
piano_mp3path = list(df_annotation[df_annotation["piano"] != 0]["mp3_path"])[:100]
vocal_mp3path = list(df_annotation[df_annotation["vocal"] != 0]["mp3_path"])[:100]
synth_mp3path = list(df_annotation[df_annotation["synth"] != 0]["mp3_path"])[:100]
classical = list(df_annotation[df_annotation["classical"] != 0]["mp3_path"])[:100]
electronic = list(df_annotation[df_annotation["electronic"] != 0]["mp3_path"])[:100]

In [65]:
target_gender = [female_mp3path, male_mp3path]
target_inst = [strings_mp3path,piano_mp3path,vocal_mp3path,synth_mp3path]
target_genre = [classical,electronic]

In [70]:
def get_target_embeddings(target_path):
    result = []
    for path_list in target_path:
        for mp3_path in path_list:
            result.append(embedding_extractor(os.path.join(dataset_path,"mp3",mp3_path),"FCN05"))
    return result

In [ ]:
gender_embeddings = get_target_embeddings(target_gender)

In [31]:
def dim_reduction(x, method='tsne', perplexity=50, n_iter=5000):
    if method == 'pca':
        trainer = PCA(n_components=2)
    else:
        trainer = TSNE(n_components=2, perplexity=perplexity, n_iter=n_iter)
    result = trainer.fit_transform(x)
    return scaler.fit_transform(result)

In [30]:
# prepare visualization
import numpy as np
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.plotting import output_notebook, output_file, reset_output

def initialize_figure(title):
    if title is None:
        title = 'Untitled'

    TOOLTIPS = [
        ("(x,y)", "($x, $y)"),
        ("entity", "@desc"),
    ]

    p = figure(title=title, tooltips=TOOLTIPS)
    p.grid.grid_line_color = None
    p.background_fill_color = "white"
    p.width = 600
    p.height = 600
    return p

def mtext(p, x, y, text, text_color, text_font_size, text_alpha):
    p.text(x, y, text=[text], text_color=text_color, text_align="center",
           text_font_size=text_font_size, text_alpha=text_alpha)

def annotation(p, coordinates, idx_to_text, text_shift=0.05,
    text_font_size='15pt', text_color="firebrick", text_alpha=1.0):

    for idx, text in enumerate(idx_to_text):
        if not text:
            continue
        x_ = coordinates[idx,0] + text_shift
        y_ = coordinates[idx,1] + text_shift
        text = idx_to_text[idx]
        mtext(p, x_, y_, text, text_color, text_font_size, text_alpha)

def draw(coordinates, idx_to_text, p=None, title=None, marker="circle",
         marker_color='orange', marker_size=5, marker_alpha=0.5):

    # prepare figure
    if p is None:
        p = initialize_figure(title)

    # prepare data source
    source = ColumnDataSource(data=dict(
        x = coordinates[:,0].tolist(),
        y = coordinates[:,1].tolist(),
        desc = idx_to_text
    ))

    # scatter plot
    p.scatter('x', 'y', marker=marker, size=marker_size, line_color='black',
              fill_color= marker_color, alpha=marker_alpha, source=source)
    return p